In [204]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf

In [205]:
data =  pd.read_csv('nifty_200/5M/ADANIPOWER.csv')

In [206]:
data.tail()

,Datetime,Open,High,Low,Close,Volume
76098,2024-08-14 15:05:00,675.00,675.75,674.40,675.20,81103
76099,2024-08-14 15:10:00,675.65,676.00,674.50,675.00,108183
76100,2024-08-14 15:15:00,675.00,675.00,671.00,672.75,170383
76101,2024-08-14 15:20:00,672.80,675.40,672.30,674.55,162340
76102,2024-08-14 15:25:00,674.55,674.60,672.25,674.50,107862


In [207]:

def calculate_macd(data, fast, slow, signal):
    data['MACD_F'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACD_{fast}_{slow}_{signal}']
    data['MACD_S'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACDs_{fast}_{slow}_{signal}']
    data['MACD_H'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACDh_{fast}_{slow}_{signal}']
    return data


In [208]:
def custom_resample(df, start='09:15', end='15:30', freq='1f'):
    df = df.between_time(start, end)
    return df.resample(freq, offset='0h15min').agg(
        {
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum',
        }
    )


In [209]:
def calculate_pivot_levels(high, low, close):
    pp = (high + low + close) / 3
    tc = (pp + high) / 2
    bc = (pp + low) / 2
    r1 = 2 * pp - low
    r2 = pp + (high - low)
    r3 = high + 2 * (pp - low)
    s1 = 2 * pp - high
    s2 = pp - (high - low)
    s3 = low - 2 * (high - pp)
    
    return pp, tc, bc, r1, r2, r3, s1, s2, s3

In [210]:
def apply_pivots(df):
    # Shift OHLC columns to get yesterday's data
    df['Prev_High'] = df['High'].shift(1)
    df['Prev_Low'] = df['Low'].shift(1)
    df['Prev_Close'] = df['Close'].shift(1)

    # Apply the function to calculate pivot levels using yesterday's data
    df[['PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3']] = df.apply(
        lambda row: calculate_pivot_levels(row['Prev_High'], row['Prev_Low'], row['Prev_Close']), 
        axis=1, 
        result_type='expand'
    )
    return df


In [211]:
def data_preprocessing(data):
    temp_Data = data.copy()
    temp_Data['Datetime'] = pd.to_datetime(temp_Data['Datetime'])
    temp_Data.set_index('Datetime', inplace=True)
    daily_data = custom_resample(temp_Data, freq='D')
    data_with_pivots = apply_pivots(daily_data)
    
    print(data.columns)
    # Add Date column to data_with_pivots
    data_with_pivots['Date'] = data_with_pivots.index.date

    data['Date'] = temp_Data.index.date
    data_with_pivots['%CPR'] = abs(data_with_pivots['TC'] - data_with_pivots['BC']) / data_with_pivots['PP']
    data_with_pivots.dropna(inplace=True)

    # Merge on the Date column
    merged_df = pd.merge(data, data_with_pivots[['Date', 'PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', '%CPR']],
                         left_on='Date', right_on='Date', how='left')
    
    merged_df['VWMA_10'] = ta.vwma(merged_df['Close'], volume=merged_df['Volume'], length=10)
    merged_df['Datetime'] = pd.to_datetime(merged_df['Datetime'])
    return merged_df.set_index('Datetime')

In [212]:
processed_Data = data_preprocessing(data).fillna(method='ffill').dropna()

Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


/var/folders/ds/wkxfdzns0q9b361mzl9c0_840000gn/T/ipykernel_7258/4146718631.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  processed_Data = data_preprocessing(data).fillna(method='ffill').dropna()


In [213]:
processed_Data[pd.isna(processed_Data['PP'])]

,Open,High,Low,Close,Volume,Date,PP,TC,BC,R1,R2,R3,S1,S2,S3,%CPR,VWMA_10
Datetime,,,,,,,,,,,,,,,,,


In [214]:
processed_Data.head(20)

,Open,High,Low,Close,Volume,Date,PP,TC,BC,R1,R2,R3,S1,S2,S3,%CPR,VWMA_10
Datetime,,,,,,,,,,,,,,,,,
2020-07-08 09:15:00,36.00,36.10,35.90,35.90,82004,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.976168
2020-07-08 09:20:00,35.90,35.95,35.85,35.85,39214,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.967341
2020-07-08 09:25:00,35.85,35.95,35.85,35.95,48933,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.964880
2020-07-08 09:30:00,35.90,36.00,35.90,36.00,39781,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.965880
2020-07-08 09:35:00,36.00,36.00,35.90,36.00,46470,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.966328
2020-07-08 09:40:00,36.00,36.00,35.90,35.90,45656,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.959961
2020-07-08 09:45:00,35.95,35.95,35.85,35.85,40137,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.944770
2020-07-08 09:50:00,35.90,35.95,35.85,35.95,24257,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.934719
2020-07-08 09:55:00,35.95,35.95,35.90,35.90,36058,2020-07-08,35.966667,36.108333,35.833333,36.233333,36.516667,36.783333,35.683333,35.416667,35.133333,0.007646,35.929726


In [215]:
(processed_Data.index)

DatetimeIndex(['2020-07-08 09:15:00', '2020-07-08 09:20:00',
               '2020-07-08 09:25:00', '2020-07-08 09:30:00',
               '2020-07-08 09:35:00', '2020-07-08 09:40:00',
               '2020-07-08 09:45:00', '2020-07-08 09:50:00',
               '2020-07-08 09:55:00', '2020-07-08 10:00:00',
               ...
               '2024-08-14 14:40:00', '2024-08-14 14:45:00',
               '2024-08-14 14:50:00', '2024-08-14 14:55:00',
               '2024-08-14 15:00:00', '2024-08-14 15:05:00',
               '2024-08-14 15:10:00', '2024-08-14 15:15:00',
               '2024-08-14 15:20:00', '2024-08-14 15:25:00'],
              dtype='datetime64[ns]', name='Datetime', length=76028, freq=None)

In [216]:
fully_merged = processed_Data

In [217]:
# Define target and stop-loss percentages
target_pct = 0.02  # 2% target
stop_loss_pct = 0.01  # 1% stop loss

# Initialize variables to store entry prices
buy_entry_price = None
short_entry_price = None



In [218]:
# Buy Entry Condition
fully_merged['Buy_Entry'] = (
    (fully_merged['%CPR'] < 0.01)& 
    (fully_merged['Close'] > fully_merged['R1']) &
    (fully_merged['VWMA_10'].shift(1) <= fully_merged['R1'].shift(1)) & 
    (fully_merged['VWMA_10'] > fully_merged['R1']) 
)

# Short Entry Condition
fully_merged['Short_Entry'] = (
    (fully_merged['%CPR'] < 0.01)& 
    (fully_merged['Close'] < fully_merged['S1']) &
    (fully_merged['VWMA_10'].shift(1) >= fully_merged['S1'].shift(1)) & 
    (fully_merged['VWMA_10'] < fully_merged['S1']) 
)


In [219]:
# Calculate entry prices and set target/stop-loss based on entry prices
for i in range(len(fully_merged)):
    if fully_merged['Buy_Entry'].iloc[i]:
        buy_entry_price = fully_merged['Close'].iloc[i]
        fully_merged.loc[fully_merged.index[i], 'Buy_Target'] = buy_entry_price * (1 + target_pct)
        fully_merged.loc[fully_merged.index[i], 'Buy_Stop_Loss'] = buy_entry_price * (1 - stop_loss_pct)
    elif fully_merged['Short_Entry'].iloc[i]:
        short_entry_price = fully_merged['Close'].iloc[i]
        fully_merged.loc[fully_merged.index[i], 'Short_Target'] = short_entry_price * (1 - target_pct)
        fully_merged.loc[fully_merged.index[i], 'Short_Stop_Loss'] = short_entry_price * (1 + stop_loss_pct)
    else:
        # Carry forward the last calculated target and stop-loss
        if buy_entry_price is not None:
            fully_merged.loc[fully_merged.index[i], 'Buy_Target'] = fully_merged['Buy_Target'].iloc[i-1]
            fully_merged.loc[fully_merged.index[i], 'Buy_Stop_Loss'] = fully_merged['Buy_Stop_Loss'].iloc[i-1]
        if short_entry_price is not None:
            fully_merged.loc[fully_merged.index[i], 'Short_Target'] = fully_merged['Short_Target'].iloc[i-1]
            fully_merged.loc[fully_merged.index[i], 'Short_Stop_Loss'] = fully_merged['Short_Stop_Loss'].iloc[i-1]


In [220]:

# Buy Exit Condition
fully_merged['Buy_Exit'] = (
    (fully_merged['Close'] < fully_merged['TC']) |
    (fully_merged['VWMA_10'].shift(1) >= fully_merged['PP'].shift(1)) & 
    (fully_merged['VWMA_10'] < fully_merged['PP'])|
    (fully_merged['Close'] >= fully_merged['Buy_Target']) |  # Target Condition
    (fully_merged['Close'] <= fully_merged['Buy_Stop_Loss'])  # Stop-Loss Condition
)

# Short Exit Condition
fully_merged['Short_Exit'] = (
    (fully_merged['Close'] > fully_merged['BC']) |
    (fully_merged['VWMA_10'].shift(1) <= fully_merged['PP'].shift(1)) & 
    (fully_merged['VWMA_10'] > fully_merged['PP'])|
    (fully_merged['Close'] <= fully_merged['Short_Target']) |  # Target Condition
    (fully_merged['Close'] >= fully_merged['Short_Stop_Loss'])  # Stop-Loss Condition
)


In [221]:
fully_merged.tail(50)

,Open,High,Low,Close,Volume,Date,PP,TC,BC,R1,...,%CPR,VWMA_10,Buy_Entry,Short_Entry,Short_Target,Short_Stop_Loss,Buy_Target,Buy_Stop_Loss,Buy_Exit,Short_Exit
Datetime,,,,,,,,,,,,,,,,,,,,,
2024-08-14 11:20:00,678.50,680.55,678.35,679.40,204787,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,680.267506,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 11:25:00,679.75,682.05,678.15,679.60,323106,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.925981,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 11:30:00,679.85,680.65,678.65,680.00,203025,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.559917,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 11:35:00,680.00,681.45,678.75,678.80,188493,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.396592,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 11:40:00,678.80,679.90,677.20,678.35,230682,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.315087,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 11:45:00,678.50,680.50,677.10,679.80,190455,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.363931,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 11:50:00,680.00,682.00,680.00,681.60,279560,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.544790,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 11:55:00,681.35,682.00,680.65,681.00,174313,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.679745,False,False,NaN,NaN,742.05,720.225,True,False
2024-08-14 12:00:00,680.55,682.00,680.50,681.50,176460,2024-08-14,691.5,697.0,688.75,697.0,...,0.011931,679.832717,False,False,NaN,NaN,742.05,720.225,True,False


In [222]:
# Initialize variables
initial_cash = 100000  # Starting with 100,000 units of currency
cash = initial_cash
position = 0  # Current position: 0 means no position, 1 means long, -1 means short
entry_price = 0
trade_history = []
risk_per_trade = 0.01  # Risk 1% of current cash balance per trade
target_pct = 0.02  # 2% target
stop_loss_pct = 0.01  # 1% stop loss
charges = 0

In [223]:
import pandas as pd
import numpy as np

# Assume fully_merged is your DataFrame with all the calculated conditions

# Define target and stop-loss percentages
target_pct = 0.02  # 2% target
stop_loss_pct = 0.01  # 1% stop loss
trade_charge = 40  # 40 units charge per trade

# Initialize variables
initial_cash = 100000  # Starting with 100,000 units of currency
cash = initial_cash
position = 0  # Current position: 0 means no position, 1 means long, -1 means short
buy_entry_price = None
short_entry_price = None
trade_history = []

# Iterate through the DataFrame
for i in range(len(fully_merged)):
    # Set up target and stop-loss for Buy Entry
    if fully_merged['Buy_Entry'].iloc[i] and position == 0:
        buy_entry_price = fully_merged['Close'].iloc[i]
        target_price = buy_entry_price * (1 + target_pct)
        stop_loss_price = buy_entry_price * (1 - stop_loss_pct)
        position = 1
        position_size = cash * 0.01 / (buy_entry_price - stop_loss_price)
        cash -= buy_entry_price * position_size + trade_charge  # Include trade charge
        charges += trade_charge
        trade_history.append({
            'Datetime': fully_merged.index[i], 
            'Type': 'Buy', 
            'Price': buy_entry_price, 
            'Size': position_size,
            'Target': target_price,
            'Stop_Loss': stop_loss_price
        })

    # Set up target and stop-loss for Short Entry
    elif fully_merged['Short_Entry'].iloc[i] and position == 0:
        short_entry_price = fully_merged['Close'].iloc[i]
        target_price = short_entry_price * (1 - target_pct)
        stop_loss_price = short_entry_price * (1 + stop_loss_pct)
        position = -1
        position_size = cash * 0.01 / (stop_loss_price - short_entry_price)
        cash += short_entry_price * position_size - trade_charge  # Include trade charge
        charges += trade_charge
        trade_history.append({
            'Datetime': fully_merged.index[i], 
            'Type': 'Short', 
            'Price': short_entry_price, 
            'Size': position_size,
            'Target': target_price,
            'Stop_Loss': stop_loss_price
        })

    # Exit conditions for Buy position
    if position == 1:
        if fully_merged['Close'].iloc[i] >= target_price:  # Target hit
            exit_price = target_price
            cash += exit_price * position_size - trade_charge  # Include trade charge
            position = 0
            profit = (exit_price - buy_entry_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (Target)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Close'].iloc[i] <= stop_loss_price:  # Stop-loss hit
            exit_price = stop_loss_price
            cash += exit_price * position_size - trade_charge  # Include trade charge
            position = 0
            profit = (exit_price - buy_entry_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (Stop Loss)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Buy_Exit'].iloc[i]:  # Regular exit condition
            exit_price = fully_merged['Close'].iloc[i]
            cash += exit_price * position_size - trade_charge  # Include trade charge
            position = 0
            profit = (exit_price - buy_entry_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (Exit)', 
                'Price': exit_price, 
                'Profit': profit
            })

    # Exit conditions for Short position
    if position == -1:
        if fully_merged['Close'].iloc[i] <= target_price:  # Target hit
            exit_price = target_price
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            position = 0
            profit = (short_entry_price - exit_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (Target)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Close'].iloc[i] >= stop_loss_price:  # Stop-loss hit
            exit_price = stop_loss_price
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            position = 0
            profit = (short_entry_price - exit_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (Stop Loss)', 
                'Price': exit_price, 
                'Profit': profit
            })
        elif fully_merged['Short_Exit'].iloc[i]:  # Regular exit condition
            exit_price = fully_merged['Close'].iloc[i]
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            position = 0
            profit = (short_entry_price - exit_price) * position_size - trade_charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (Exit)', 
                'Price': exit_price, 
                'Profit': profit
            })

    # Check for end of day (intraday close)
    if fully_merged.index[i].time() == pd.to_datetime("15:15").time() and position != 0:
        exit_price = fully_merged['Close'].iloc[i]
        if position == 1:  # If in a long position
            cash += exit_price * position_size - trade_charge  # Include trade charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Sell (End of Day)', 
                'Price': exit_price, 
                'Profit': (exit_price - buy_entry_price) * position_size - trade_charge
            })
        elif position == -1:  # If in a short position
            cash -= exit_price * position_size + trade_charge  # Include trade charge
            trade_history.append({
                'Datetime': fully_merged.index[i], 
                'Type': 'Cover (End of Day)', 
                'Price': exit_price, 
                'Profit': (short_entry_price - exit_price) * position_size - trade_charge
            })
        position = 0  # Reset position to zero




# Final Portfolio Value
final_value = cash

# Convert trade history to DataFrame
trade_history_df = pd.DataFrame(trade_history)

# Calculate KPIs
total_trades = len(trade_history_df) // 2  # Assuming Buy/Sell pairs or Short/Cover pairs
winning_trades = trade_history_df[trade_history_df['Profit'] > 0]
losing_trades = trade_history_df[trade_history_df['Profit'] <= 0]

total_profit = trade_history_df['Profit'].sum()
win_rate = len(winning_trades) / total_trades if total_trades > 0 else 0
average_profit = trade_history_df['Profit'].mean()
profit_factor = winning_trades['Profit'].sum() / abs(losing_trades['Profit'].sum()) if len(losing_trades) > 0 else np.inf
max_drawdown = (trade_history_df['Profit'].cumsum().cummax() - trade_history_df['Profit'].cumsum()).max()
sharpe_ratio = trade_history_df['Profit'].mean() / trade_history_df['Profit'].std() * np.sqrt(252) if trade_history_df['Profit'].std() > 0 else np.nan
sortino_ratio = trade_history_df['Profit'].mean() / trade_history_df[trade_history_df['Profit'] < 0]['Profit'].std() * np.sqrt(252) if trade_history_df[trade_history_df['Profit'] < 0]['Profit'].std() > 0 else np.nan

# Output Results
print(f"Initial Cash: {initial_cash}")
print(f"Final Portfolio Value: {final_value}")
print(f"Total Profit: {total_profit}")
print(f"Return Percentage: {((final_value - initial_cash) / initial_cash) * 100:.2f}%")
print(f"Number of Trades: {total_trades}")
print(f"Win Rate: {win_rate:.2f}")
print(f"Average Profit/Loss per Trade: {average_profit:.2f}")
print(f"Profit Factor: {profit_factor:.2f}")
print(f"Max Drawdown: {max_drawdown:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Sortino Ratio: {sortino_ratio:.2f}")



Initial Cash: 100000
Final Portfolio Value: 92921.89218088068
Total Profit: -2838.107819119252
Return Percentage: -7.08%
Number of Trades: 106
Win Rate: 0.34
Average Profit/Loss per Trade: -26.77
Profit Factor: 0.93
Max Drawdown: 8814.45
Sharpe Ratio: -0.44
Sortino Ratio: -1.15


In [224]:
trade_history_df

,Datetime,Type,Price,Size,Target,Stop_Loss,Profit
0,2020-07-10 09:30:00,Short,35.3500,2828.854314,34.643,35.7035,NaN
1,2020-07-10 15:15:00,Cover (End of Day),34.9000,NaN,NaN,NaN,1232.984441
2,2020-07-14 11:40:00,Short,34.3500,2945.938412,33.663,34.6935,NaN
3,2020-07-14 12:40:00,Cover (Exit),34.5500,NaN,NaN,NaN,-629.187682
4,2020-07-14 14:10:00,Buy,36.0000,2792.327688,36.720,35.6400,NaN
...,...,...,...,...,...,...,...
207,2024-07-08 15:15:00,Cover (End of Day),697.0000,NaN,NaN,NaN,800.243761
208,2024-07-16 09:15:00,Buy,727.0500,129.054148,741.591,719.7795,NaN
209,2024-07-16 09:25:00,Sell (Stop Loss),719.7795,NaN,NaN,NaN,-978.288184
210,2024-07-30 09:45:00,Buy,727.5000,127.574612,742.050,720.2250,NaN


In [231]:
def plot_trades_on_chart(data, trade_history_df, file_name):
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])

    # Add markers for trades
    for i, trade in trade_history_df.iterrows():
        if trade['Type'] == 'Buy':
            fig.add_trace(go.Scatter(x=[trade['Datetime']],
                                     y=[trade['Price']],
                                     mode='markers',
                                     marker=dict(symbol='triangle-up', color='green', size=10),
                                     name='Buy Entry'))
        elif trade['Type'] == 'Sell' or trade['Type'] == 'Sell (End of Day)':
            fig.add_trace(go.Scatter(x=[trade['Datetime']],
                                     y=[trade['Price']],
                                     mode='markers',
                                     marker=dict(symbol='triangle-down', color='red', size=10),
                                     name='Sell Exit'))
        elif trade['Type'] == 'Short':
            fig.add_trace(go.Scatter(x=[trade['Datetime']],
                                     y=[trade['Price']],
                                     mode='markers',
                                     marker=dict(symbol='triangle-down', color='orange', size=10),
                                     name='Short Entry'))
        elif trade['Type'] == 'Cover' or trade['Type'] == 'Cover (End of Day)':
            fig.add_trace(go.Scatter(x=[trade['Datetime']],
                                     y=[trade['Price']],
                                     mode='markers',
                                     marker=dict(symbol='triangle-up', color='blue', size=10),
                                     name='Cover Exit'))

    fig.update_layout(
        title=f'{file_name} - Trade Visualization',
        xaxis_title='Datetime',
        yaxis_title='Price',
        xaxis_rangeslider_visible=True,  # Enable range slider for zooming in on the chart
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),  # Add range slider
            type="date"
        ),
        yaxis=dict(fixedrange=False),  # Allow scaling on the y-axis
        dragmode='zoom',  # Enable zooming by default
        template='plotly_dark'  # Optional: use a dark theme for better visibility
    )

    # Save the plot as an HTML file
    plot_dir = 'Backtest/Plots'
    os.makedirs(plot_dir, exist_ok=True)
    fig.write_html(os.path.join(plot_dir, f'{file_name}_trade_visualization.html'))
    print(f"Trade visualization saved to '{plot_dir}/{file_name}_trade_visualization.html'")

In [232]:
plot_trades_on_chart(fully_merged,trade_history_df, 'ADANIPOWER')

Trade visualization saved to 'Backtest/Plots/ADANIPOWER_trade_visualization.html'
